In [1]:
import sys
print(sys.path)

['/Users/raven.mukherjee/edi-sol-accelerator/x12-edi-parser/databricksx12/hls/test-notebooks', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/raven.mukherjee/edi-sol-accelerator/edi-parse-env/lib/python3.12/site-packages']


In [2]:
from databricksx12.edi import *
x =  EDIManager(EDI(open("/Users/raven.mukherjee/solution_accelerators/x12-edi-parser/sampledata/837/CHPW_Claimdata.txt", "rb").read().decode("utf-8")))

import json
print(json.dumps(x.flatten(x.data), indent=4))

{
    "EDI.sender_tax_id": "ZZ",
    "list": [
        {
            "FunctionalGroup.receiver": "123456789",
            "FunctionalGroup.sender": "CLEARINGHOUSE",
            "FunctionalGroup.transaction_datetime": "20180508:0833",
            "FunctionalGroup.transaction_type": "222",
            "list": [
                {
                    "Transaction.transaction_type": "222"
                },
                {
                    "Transaction.transaction_type": "222"
                },
                {
                    "Transaction.transaction_type": "222"
                },
                {
                    "Transaction.transaction_type": "222"
                },
                {
                    "Transaction.transaction_type": "222"
                }
            ]
        }
    ]
}


In [3]:
sample_data_837i = open("/Users/raven.mukherjee/solution_accelerators/x12-edi-parser/sampledata/837/CC_837I_EDI.txt", "rb").read().decode("utf-8")

In [4]:
# from databricksx12.edi import *

# class extend_transaction(EDI):
#     def __init__(self, data, delim_cls=AnsiX12Delim):
#         super().__init__(data, delim_cls)

#     @property
#     def full_transaction(self):
#         transaction_start_indexes = [i for i, segment in enumerate(self.data) if segment.segment_name() == "ST"]
#         transaction_end_indexes = [i for i, segment in enumerate(self.data) if segment.segment_name() == "SE"]

#         transactions = []
#         for start, end in zip(transaction_start_indexes, transaction_end_indexes):
#             transaction_segments = self.data[start:end+1]
#             transactions.append(transaction_segments)
#         return transactions

#     @property
#     def claim_identifier(self):
#         transactions = self.full_transaction
#         claim_identifiers = []

#         for transaction_segments in transactions:
#             claim_id = None
#             for segment in transaction_segments:
#                 if segment.segment_name() == "BHT":
#                     claim_id = segment.element(3) #confirm
#                     break
#             claim_identifiers.append(claim_id)

#         return claim_identifiers

#     @property
#     def header_billing_amount(self):
#         transactions = self.full_transaction
#         billing_headers = []

#         for transaction_segments in transactions:
#             for segment in transaction_segments:
#                 if segment.segment_name() == "CLM":
#                     bill_header = segment.element(1)
#                     billing_headers.append(bill_header)
#                     break  # one CLM segment per transaction?

#         return billing_headers

#     @property
#     def billed_amount(self):
#         transactions = self.full_transaction
#         billed_amounts = []

#         for transaction_segments in transactions:
#             for segment in transaction_segments:
#                 if segment.segment_name() == "CLM":
#                     billed_amount = segment.element(2)  # Billed amount is the second element
#                     billed_amounts.append(billed_amount)
#                     break

#         return billed_amounts
    
#     # @property
#     # def subscriber(self):
#     #     transactions = self.full_transaction




In [5]:
# # use raw EDI data
# edi_object = extend_transaction(sample_data_837i)

# # call  different vars
# transactions = edi_object.full_transaction
# claim_ids = edi_object.claim_identifier
# header = edi_object.header_billing_amount
# billed_amount = edi_object.billed_amount

In [12]:
#  identify elements functionally!

from databricksx12.edi import *

class extend_transaction(EDI):
    def __init__(self, data, delim_cls=AnsiX12Delim):
        super().__init__(data, delim_cls)

        # Use map and lambda to populate billed amounts and subscribers
        self.billed_amounts = list(map(lambda x: x.element(2), self.segments_by_name("CLM")))
        self.subscribers = list(map(lambda x: x.element(4), self.segments_by_name("SBR")))



In [13]:
sample_data_chpw_claimdata = open("/Users/raven.mukherjee/edi-sol-accelerator/x12-edi-parser/sampledata/837/CHPW_Claimdata.txt", "rb").read().decode("utf-8")
# use raw EDI data
edi_object = extend_transaction(sample_data_chpw_claimdata)
billed_amounts = edi_object.billed_amounts
subscribers = edi_object.subscribers


In [14]:
billed_amounts

['20', '50.1', '11.64', '234', '20']

In [15]:
subscribers

['COMMUNITY HLTH PLAN OF WASH',
 'COMMUNITY HLTH PLAN OF WASH',
 'COMMUNITY HLTH PLAN OF WASH',
 'COMMUNITY HLTH PLAN OF WASH',
 'COMMUNITY HLTH PLAN OF WASH']